In [1]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime
import json 

In [ ]:
file = "./PA/pa_final.geojson"    

precincts = gp.read_file(file)

In [ ]:
#reader = csv.DictReader('../data/NY_county.csv')
reader = csv.reader(open('./PA_county.csv', 'r'))
d = {}
for row in reader:
    #id = row['County FIPS']
    key = row[3]
    value = row[5]
    d[key] = value
    
#print (d)

In [ ]:
precincts.head()

In [ ]:
#Rename columns
precincts = precincts.drop (columns = ['STATEFP10', 'VTDI10', 'NAME10', 'NAMELSAD10', 'LSAD10',
                         'MTFCC10','FUNCSTAT10','ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', ])
                       # 'cartodb_id', 'other', 'other_perc', 'male', 'male_perce', 'female',
                        # 'female_per', 'total_regi', 'registered', 'id', 'data', 'dem_rep_di'])

precincts.rename(columns = {"GEOID10":"id"}, inplace = True)
#df.rename(columns = {"GEOID10":"canonical"}, inplace = True)
#precincts.rename(columns = {"NEIGHBORS":"neighbors"}, inplace = True)
#df.rename(columns = {"INTPTLON10":"centerY"}, inplace = True)

#Add type columns
precincts.insert(2, 'type', 'ORIGINAL')
precincts.insert(3, 'centerX', None)
precincts.insert(4, 'centerY', None)
precincts.insert(5, 'area', None)
precincts.insert(6, 'neighbors', None)
precincts.insert(7, 'canonical', None)

#precincts = precincts.drop (columns = ['SUM'])
precincts.head()

In [ ]:
new_precinct = pd.

In [ ]:
#Canonical name
state = 'PA'
for i, row in precincts.iterrows():   
    id = str(precincts.at[i, 'VTDST10']).zfill(5)
    countyId = precincts.at[i, 'COUNTYFP10']
    #print (countyId)
    canonicalId = state + ' ' + d[countyId] + ' ' + id
    precincts.at[i, 'canonical'] = canonicalId
    precincts.at[i, 'id'] = '42' + str(precincts.at[i, 'COUNTYFP10']) + id
    #print (canonicalId)

precincts.head()

In [103]:
#ADD area and centers
for i, row in precincts.iterrows():
    precincts.at[i, 'area'] = row.geometry.area
    #bounds = row.geometry.bounds
    #centerX = round((bounds[0] + bounds[2]) / 2, 6)
    #centerY = round((bounds[1] + bounds[3]) / 2, 6)
    #precincts.at[i, 'centerX'] = centerY
    #precincts.at[i, 'centerY'] = centerX    

precincts.head()

,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,ORIGINAL,41.157814,-80.479783,0.004278359631999892,"4207300790, 4208500950, 4208501040, 4208500425...",PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51877 41.187..."
1,ORIGINAL,41.680613,-80.04085000000001,0.0005506003439999464,"4203900150, 4203900330, 4203900340, 4203900660",PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.04599 41.684..."
2,ORIGINAL,41.743054,-79.840985,0.007933951458499875,"4203900030, 4203900070, 4203900400, 4203900360...",PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.83310 41.775..."
3,ORIGINAL,41.803153,-80.462641,0.01028330178850014,"4203900090, 4203900420, 4204900040",PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.40589 41.849..."
4,ORIGINAL,41.812444,-79.836173,0.01073166413550007,"4203900010, 4203900400, 4203900350, 4204901060...",PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.76203 41.850..."


In [ ]:
precincts = precincts.drop(columns = ['COUNTYFP10', 'VTDST10'])
precincts.head()

In [ ]:
precincts.to_file("./PA/PA_new.geojson", driver='GeoJSON')
precincts.head()

In [ ]:
mappingPrecinct = pd.read_excel('./PA/PA_GEOID_to_PRECINCT_C.xlsx')
mappingPrecinct.head()

In [ ]:
for i, row in mappingPrecinct.iterrows():
    geoid = row['GEOID10']
    original_id = geoid[5:].zfill(5)
    precinct_c = str(row['PRECINCT_C']).zfill(5)
    county_fips = geoid[:5]
    mappingPrecinct.at[i, 'GEOID10'] = county_fips + original_id
    mappingPrecinct.at[i, 'PRECINCT_C'] = county_fips + precinct_c
mappingPrecinct.head()

In [ ]:
duplicateRowsDF = mappingPrecinct[mappingPrecinct['PRECINCT_C'].duplicated()]
duplicateRowsDF.shape

In [ ]:
mappingPrecinct.drop_duplicates(subset ="PRECINCT_C", 
                     keep = False, inplace = True) 

In [ ]:
mappingDict = {}
for i, row in mappingPrecinct.iterrows():
    mappingDict[row['GEOID10']] = row['PRECINCT_C']
print(mappingDict)

In [ ]:
for i, row in precincts.iterrows():
    if row['id'] in mappingDict.keys():
        precincts.at[i, 'id'] = mappingDict[row['id']]

In [ ]:
file = "./PA/pa_final.geojson"    

precincts_new = gp.read_file(file)
precincts_new.head()

In [ ]:
precinct_dupl = precincts_new[precincts_new['id'].duplicated()]
precinct_dupl.shape

In [61]:
### GENERATE NEIGHBORS

#file = "./PA/PA.geojson"  
#state_boundary = gp.read_file(file) # open file

precincts["neighbors"] = None  # add NEIGHBORS column

for index, row in precincts.iterrows():
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
        
    neighbors = precincts[~precincts.geometry.disjoint(row.geometry)].id.tolist()
    # remove own name from the list
    neighbors = [ str(name) for name in neighbors if row.id != name ]
    #print(neighbors)
    # add names of neighbors as NEIGHBORS value
    precincts.at[index, 'neighbors'] = ", ".join(neighbors)

precincts.head()


09:05:14 0
09:05:36 1000
09:05:57 2000
09:06:19 3000
09:06:40 4000
09:07:02 5000
09:07:24 6000
09:07:46 7000
09:08:08 8000
09:08:29 9000


,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,ORIGINAL,41.157814,-80.479783,0.004278359631999892,"4207300790, 4208500950, 4208501040, 4208500425...",PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51877 41.187..."
1,ORIGINAL,41.680613,-80.04085000000001,0.0005506003439999464,"4203900150, 4203900330, 4203900340, 4203900660",PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.04599 41.684..."
2,ORIGINAL,41.743054,-79.840985,0.007933951458499875,"4203900030, 4203900070, 4203900400, 4203900360...",PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.83310 41.775..."
3,ORIGINAL,41.803153,-80.462641,0.01028330178850014,"4203900090, 4203900420, 4204900040",PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.40589 41.849..."
4,ORIGINAL,41.812444,-79.836173,0.01073166413550007,"4203900010, 4203900400, 4203900350, 4204901060...",PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.76203 41.850..."


In [87]:
precincts.to_file("./PA/PA_precinct_c.geojson", driver='GeoJSON')

In [ ]:
def create_neighbors_df(neighbors, df):
    neighbors = neighbors.split(',')
    neighbors_df = df[df['id'].isin(neighbors)][['id', 'geometry']]
    return neighbors_df 

In [ ]:
#INTERSECTIONS

file = "./PA/PA.geojson"  
precincts = gp.read_file(file) # open file
precincts = precincts.drop(columns=['type', 'centerX', 'centerY', 'canonical'])
precincts.head()

In [ ]:
AREA_THRESHOLD = 1e-4
intersections = gp.GeoDataFrame(crs = precincts.crs)

for index, row in precincts.iterrows():
    #For progress
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
    neighbors = create_neighbors_df(row['neighbors'], precincts)
    #print(neighbors)
    # check if intersection occured
    overlaps = neighbors[neighbors.geometry.overlaps(row.geometry)]['id'].tolist()
    if len(overlaps) > 0:
        # compare the area with threshold
        for y in overlaps:
            temp_area = gp.overlay(precincts.loc[precincts.id == y,], precincts.loc[precincts.id == row.id], how='intersection')
            temp_area = temp_area.loc[temp_area.geometry.area >= AREA_THRESHOLD]
            
            if temp_area.shape[0] > 0:
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections], ignore_index=True),crs = precincts.crs)
                intersections['sorted'] = intersections.apply(lambda y: sorted([y['id_1'],y['id_2']]), axis=1)
                intersections['sorted'] = intersections.sorted.apply(lambda y: ''.join(y))
                intersections = intersections.drop_duplicates('sorted')
                intersections = intersections.reset_index()[['id_1','id_2','geometry']]
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections],ignore_index=True),crs=precincts.crs)

In [170]:
### GAPS

file = "./PA/PA_precinct_c.geojson"
precincts = gp.read_file(file)

precincts_diss = precincts[['geometry', 'type']]
precincts_diss = precincts_diss.dissolve(by='type')
#precincts_diss = df.drop (columns = ['type', 'id', 'canonical', 'centerX', 'centerY', 'area'])

precincts_diss.head()

,geometry
type,
ORIGINAL,"MULTIPOLYGON (((-76.33238 39.72130, -76.33703 ..."


In [171]:
interior = precincts_diss.interiors.values.tolist()
print(interior)
gaps = []
for i in interior:
    if i == []:
        continue
    #print(i[0].coords[:])    
    gaps.append(Polygon(i[0]))
    
gaps = gp.GeoDataFrame(geometry = gaps, crs = precincts.crs)
gaps['feature_touches'] = gaps.geometry.apply(lambda y: precincts.loc[precincts.touches(y)]['id'].tolist())
gaps.head()

[None]


C:\ProgramData\Anaconda3\lib\site-packages\geopandas\array.py:575: UserWarning: Only Polygon objects have interior rings. For other geometry types, None is returned.
  "Only Polygon objects have interior rings. For other "


TypeError: 'NoneType' object is not subscriptable

In [95]:
## MAP COVERAGE ERROR

file_states = "./States/states_boundary.geojson"
state_boundary = gp.read_file(file_states)
state_boundary.head()

,id,centerX,centerY,area,geometry
0,PA,-77.6048035,41.1179355,12.7471968513065,"MULTIPOLYGON (((-80.51935 41.84956, -80.51922 ..."
1,KY,-85.76960750000001,37.82234,10.668081506716,"MULTIPOLYGON (((-88.06133 37.50533, -88.06368 ..."
2,NY,-75.77002999999999,42.7460485,15.58035125575651,"MULTIPOLYGON (((-79.31214 42.68680, -79.14347 ..."


In [97]:
file_precincts = "./PA/PA_final.geojson"
precinct_boundary = gp.read_file(file_precincts)
precinct_boundary.head(2)

,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,ORIGINAL,41.157814,-80.479783,0.004278359631999892,"4207300790, 4208500950, 4208501040, 4208500425...",PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51877 41.187..."
1,ORIGINAL,41.680613,-80.04085000000001,0.0005506003439999464,"4203900150, 4203900330, 4203900340, 4203900660",PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.04599 41.684..."


In [107]:
#Check difference between two areas
STATE = 'PA'
state = state_boundary[state_boundary['id'] == STATE]
state_area = state_boundary.at[1, 'area']
precinct_area = 0
for i, row in precinct_boundary.iterrows():
    precinct_area += float(row['area'])
#print(precinct_area)
print("Difference: ", float(state_area) - float(precinct_area))

Difference:  -2.0792022069440126


In [108]:
precinct_dissolved = precinct_boundary[['geometry', 'type']]
precinct_dissolved = precinct_dissolved.dissolve(by='type')
precinct_dissolved.head()

,geometry
type,
ORIGINAL,"MULTIPOLYGON (((-76.33238 39.72130, -76.33703 ..."


In [109]:
difference = gp.overlay(state, precinct_dissolved, how = 'difference')

In [117]:
difference.to_file("./PA/PA_coverage_error.geojson", driver='GeoJSON')

In [172]:
THRESHOLD = 5e-4

add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

ERROR_COVERAGE_TYPE = 'COVERAGE'
STATE = 'PA'
coverage_error_data = []

for multipolygon in difference.geometry:
    for polygon in multipolygon:
        #print(polygon.area)
        if polygon.area > THRESHOLD:
            bounds = polygon.bounds
            centerX = round((bounds[0] + bounds[2]) / 2, 6)
            centerY = round((bounds[1] + bounds[3]) / 2, 6)
            coverage_row = (ERROR_COVERAGE_TYPE, None, centerX, centerY, STATE)
            coverage_error_data.append(coverage_row)
            
#cursor.executemany(add_error_querry, coverage_error_data)
#mydb.commit()           
print(coverage_error_data)

[('COVERAGE', None, -77.658383, 39.726272, 'PA'), ('COVERAGE', None, -77.86815, 41.78234, 'PA'), ('COVERAGE', None, -80.299713, 42.360511, 'PA'), ('COVERAGE', None, -80.5185, 40.077003, 'PA')]


In [198]:
### ANOMALOUS DATA
file = "./demographic_PA/precinct_demographic_PA.json"
demographics = pd.read_json(file)
demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian
0,4208500960,1700,1670,0,0,10,0,0
1,4203900040,158,154,0,0,3,0,0
2,4203900010,801,798,0,16,0,0,0
3,4203900020,771,766,2,0,0,0,0
4,4203900030,2092,2044,9,26,0,5,0


In [199]:
file = "./election/PA_president2016.json"
president = pd.read_json(file)  
file = "./election/PA_congress2016.json"
congress2016 = pd.read_json(file)
file = "./election/PA_congress2018.json"
congress2018 = pd.read_json(file)

In [200]:
print(president.shape)
print(congress2016.shape)
print(congress2018.shape)
print(demographics.shape)

(9256, 4)
(9256, 4)
(9256, 4)
(9256, 8)


In [206]:
anomalous_data = []
for index, row in demographics.iterrows():    
    anomalous = False
    
    if row['total'] <= 0:
        anomalous = True       
        
    if president.at[index, 'democrat'] == 0 and president.at[index, 'republican'] == 0:
        anomalous = True
    
    if congress2016.at[index, 'democrat'] == 0 and congress2016.at[index, 'republican'] == 0:        
        anomalous = True
        
    if congress2018.at[index, 'democrat'] == 0 and congress2018.at[index, 'republican'] == 0:        
        anomalous = True
        
    #if president.at[index, 'democrat'] + president.at[index, 'republican'] > row['total']:
        #anomalous = True
        
    if anomalous:
        anomalous_data.append(row['id'])

print(len(anomalous_data))

72


In [210]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [145]:
file = "./PA/update_pa_precinct_c.geojson"
precincts = gp.read_file(file)
precincts.head()

,type,centerX,centerY,neighbors,canonical,precinct_c,id,geometry
0,ORIGINAL,41.157814,-80.479783,"4207300790, 4208500950, 4208501040, 4208500425...",PA Mercer 00960,4208500960,4208500960,"MULTIPOLYGON (((-80.51922 41.18555, -80.51877 ..."
1,ORIGINAL,41.680613,-80.04085000000001,"4203900150, 4203900330, 4203900340, 4203900660",PA Crawford 00040,4203900040,4203900040,"MULTIPOLYGON (((-80.05844 41.68391, -80.04599 ..."
2,ORIGINAL,41.743054,-79.840985,"4203900030, 4203900070, 4203900400, 4203900360...",PA Crawford 00010,4203900010,4203900010,"MULTIPOLYGON (((-79.90979 41.77394, -79.83310 ..."
3,ORIGINAL,41.803153,-80.462641,"4203900090, 4203900420, 4204900040",PA Crawford 00020,4203900020,4203900020,"MULTIPOLYGON (((-80.51935 41.84956, -80.40589 ..."
4,ORIGINAL,41.812444,-79.836173,"4203900010, 4203900400, 4203900350, 4204901060...",PA Crawford 00030,4203900030,4203900030,"MULTIPOLYGON (((-79.90998 41.85069, -79.76203 ..."


In [146]:
# ADD PRECINCT
add_precinct_querry = ("INSERT INTO mhaczynska.precinct "
               "(GEO_ID, NAME, STATE_NAME, CENTER_X, CENTER_Y, NEIGHBORS, PRECINCT_TYPE, PRECINCT_GEOJSON) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")

update_precinct_querry = ("UPDATE mhaczynska.precinct SET PRECINCT_GEOJSON = %s WHERE GEO_ID = %s")

state = 'PA'
#precinct_data = []
for index, row in precincts.iterrows():
    geometry_str = row.geometry.__geo_interface__
    geometry_str = json.dumps(geometry_str)     
    #precinct_data = (row['id'], row['canonical'], state, row['centerX'], row['centerY'], row['neighbors'], row['type'], geometry_str)
    #precinct_data.append(precinct_row)
    update_precinct = (geometry_str, row['id'])
    cursor.execute(update_precinct_querry, update_precinct)
    mydb.commit()
    

In [162]:
#ADD NEIGHBORS
add_neighbor_querry = ("INSERT INTO mhaczynska.neighbors "
               "(PRECINCT, NEIGHBOR_GEOID) "
               "VALUES (%s, %s)")

neighbor_data = []
for index, row in precincts.iterrows():
    neighbors = row['neighbors']
    neighbors = neighbors.split(",")
    for neighbor in neighbors:
        neighbor_row = (row['id'], neighbor.strip())
        neighbor_data.append(neighbor_row)        
#print(neighbor_data)

cursor.executemany(add_neighbor_querry, neighbor_data)
mydb.commit()

In [80]:
precinctC_mapping = {}
reader = csv.reader(open('./PA/PA_GEOID_to_PRECINCT_C.csv', 'r'))
for row in reader:
    #id = row['County FIPS']
    county_fips = str(row[0])[:5]
    key = str(row[0])[5:].zfill(5)
    key = county_fips + key
    value = str(row[1]).zfill(5)
    value = county_fips + value
    #print(key, value)
    precinctC_mapping[key] = value

In [ ]:
precincts.insert(6, 'precinct_c', None)

for i, row in precincts.iterrows():
    if row['id'] in precinctC_mapping.keys():
        precincts.at[i, 'precinct_c'] = precinctC_mapping[row['id']]
precincts.head()

In [138]:
## OVERLAP 
add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

file = './PA/update_pa_points_of_intersection.geojson'
intersections = gp.read_file(file)
intersections_data = []
ERROR_INTERSECTIONS_TYPE = 'OVERLAP'
STATE = 'PA'
for index, row in intersections.iterrows():
    point = list(row.geometry.coords)
    intersection_row = (ERROR_INTERSECTIONS_TYPE, row['id'], point[0][1], point[0][0], STATE)
    intersections_data.append(intersection_row)

cursor.executemany(add_error_querry, intersections_data)
mydb.commit()    
#print(intersections_data)

In [157]:
#ENCLOSED PRECINCTS

add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

update_enclosed_error_querry = "UPDATE mhaczynska.data_error SET STATE = %s WHERE PRECINCT = %s"

ERROR_ECLOSED_TYPE = 'ENCLOSED'
ERROR_MULTIPOLYGON_TYPE = 'MULTIPOLYGON'
STATE = 'PA'
enclosed_error_data = []
multipolygon_error_data = []
for index, row in precincts.iterrows():    
    if  len(row.geometry) > 1:
        multipolygon_row = (ERROR_MULTIPOLYGON_TYPE, row['id'], row['centerX'], row['centerY'], STATE)
        multipolygon_error_data.append(multipolygon_row)
    for i, polygon in enumerate(row.geometry):
            #if i != 0:
        interior_list = list(polygon.interiors)
        if len(interior_list) > 0:
            bounds = interior_list[0].bounds
            centerX = round((bounds[0] + bounds[2]) / 2, 6)
            centerY = round((bounds[1] + bounds[3]) / 2, 6)
            enclosed_row = (ERROR_ECLOSED_TYPE, row['id'], centerX, centerY, STATE)
            enclosed_error_data.append(enclosed_row)

#print(enclosed_error_data)           
#print(multipolygon_error_data)            
cursor.executemany(add_error_querry, enclosed_error_data)
mydb.commit()
cursor.executemany(add_error_querry, multipolygon_error_data)
mydb.commit()

In [211]:
add_error_querry = ("INSERT INTO mhaczynska.data_error "
               "(ERROR_TYPE, PRECINCT, CENTER_X, CENTER_Y, STATE) "
               "VALUES (%s, %s, %s, %s, %s)")

STATE = 'PA'
ERROR_ANOMALOUS_TYPE = 'ANOMALOUS'
anomalous = []

for precinct_id in anomalous_data:
    centerX = precincts[precincts['id'] == precinct_id]['centerX'].values[0]
    centerY = precincts[precincts['id'] == precinct_id]['centerY'].values[0]
    anomalous_row = (ERROR_ANOMALOUS_TYPE, precinct_id, centerX, centerY, STATE)
    anomalous.append(anomalous_row)

cursor.executemany(add_error_querry, anomalous)
mydb.commit()    
#print(anomalous)

In [212]:
cursor.close()
mydb.close()